In [1]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import os
import torch
import random
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from IPython.display import Image

SEED = 24535


def seed_everything(seed=SEED):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    # torch.backends.cudnn.deterministic = True


seed_everything()


In [2]:
pd.set_option('display.max_rows', 500)


In [3]:
train = pd.read_csv('data_set_ALL_AML_train.csv')


In [4]:
# len([c for c in test.columns if 'call' in c])

In [5]:
# # import pandas.rpy.common as com
# import seaborn as sns
# %matplotlib inline

# # # load the R package ISLR
# # infert = com.importr("ISLR")

# # # load the Auto dataset
# # auto_df = com.load_data('Auto')

# # calculate the correlation matrix
# corr = train.corr()

# # plot the heatmap
# sns.heatmap(corr, 
#         xticklabels=corr.columns,
#         yticklabels=corr.columns)

In [6]:
def get_dummies(df):
    res = []
    for c in df.columns:
        # if 'call' not in c:
        #     continue
        res.append(pd.get_dummies(df[c]))
    return pd.concat(res, axis=1)

# get/_dummies(train)

In [7]:
expressions_by_letter = {c: [] for c in ['A', 'P', 'M']}

for c in train.columns:
    if 'call.' not in c:
        continue
    
    person_col = c.split('.')[-1]
    for e, l in zip(train[person_col], train[c]):
        expressions_by_letter[l].append(e)


In [8]:
pd.Series(expressions_by_letter['A']).describe()

count    182889.000000
mean         94.598024
std         655.889105
min      -28400.000000
25%         -73.000000
50%          43.000000
75%         214.000000
max       41911.000000
dtype: float64

In [9]:
pd.Series(expressions_by_letter['M']).describe()

count     4245.000000
mean       455.317314
std        782.500963
min      -1618.000000
25%        109.000000
50%        270.000000
75%        563.000000
max      14013.000000
dtype: float64

In [10]:
pd.Series(expressions_by_letter['P']).describe()

count    76639.000000
mean      1944.843148
std       4094.395703
min      -4125.000000
25%        230.000000
50%        581.000000
75%       1442.000000
max      61228.000000
dtype: float64

In [11]:
# train

In [12]:
test = pd.read_csv('data_set_ALL_AML_independent.csv')


In [13]:
train.shape, test.shape


((7129, 78), (7129, 70))

In [14]:
def get_person_columns(df):
    return [c for c in df.columns if c.isdigit()]


In [15]:
target_orig = pd.read_csv('actual.csv')
target_orig.shape


(72, 2)

In [16]:
train_expressions_cols = get_person_columns(
    train)  # list(map(str,range(1,38)))
test_expressions_cols = get_person_columns(
    test)  # list(map(str,range(39, 63)))
# expressions_cols


In [17]:
# train_expressions_cols


In [18]:
# train['Gene Accession Number']


In [19]:
train_exp = pd.DataFrame(train[train_expressions_cols]).set_index(
    train['Gene Accession Number'])
test_exp = pd.DataFrame(test[test_expressions_cols]).set_index(
    test['Gene Accession Number'])


In [20]:
import plotly.graph_objects as go

# fig = go.Figure(data=[go.Histogram(x=exp.values.reshape(-1))])
# fig.show()


In [49]:

def get_dummies(df):
    call_df = df[[c for c in df.columns if 'call' in c]].T
    return pd.DataFrame(pd.get_dummies(call_df.values.reshape(-1)).values.reshape((len(call_df), -1))).set_index(call_df.index)

get_dummies(train)

,0,1,2,3,4,5,6,7,8,9,...,21377,21378,21379,21380,21381,21382,21383,21384,21385,21386
call,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
call.1,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
call.2,1,0,0,1,0,0,1,0,0,1,...,1,1,0,0,1,0,0,1,0,0
call.3,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
call.4,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
call.5,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
call.6,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,0,1,1,0,0
call.7,1,0,0,1,0,0,1,0,0,1,...,1,1,0,0,0,0,1,1,0,0
call.8,1,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,1,0,0
call.9,1,0,0,1,0,0,1,0,0,0,...,1,1,0,0,0,1,0,1,0,0


In [50]:
X_train = train_exp.rename(
    {c: f"person{c}" for c in train_exp.columns}, axis='columns').T.sample(frac=1)
X_train = X_train.join(get_dummies(train))
X_test = test_exp.rename(
    {c: f"person{c}" for c in test_exp.columns}, axis='columns').T
X_test = X_test.join(get_dummies(test))
val_size = 16
X_val = X_train[-val_size:]
X_train = X_train[:-val_size]


In [51]:
X_train.shape, X_val.shape, X_test.shape


((22, 28516), (16, 28516), (34, 28516))

In [21]:
target = pd.Series(target_orig.cancer.values, index=[
                   f"person{n}" for n in target_orig.patient]).rename("cancer")
target_one_hot = (target == "ALL").astype(int)
target_one_hot.value_counts()
Y_train = target_one_hot.loc[X_train.index]
Y_val = target_one_hot.loc[X_val.index]
Y_test = target_one_hot.loc[X_test.index]


In [22]:
import phik
from phik import resources, report


In [27]:
from tqdm import tqdm
phik_cors = {}
for c in tqdm(X_train.columns):
    col_cor = pd.concat([X_train[c], Y_train], axis=1).phik_matrix(verbose=False)[
        'cancer'].iloc[:-1]
    phik_cors[c] = col_cor[0]
phik_cors = pd.Series(phik_cors)


# [['M84526_at']]


100%|██████████| 7129/7129 [02:46<00:00, 42.82it/s]


In [29]:
phik_cors.sort_values()[-10:]


U78190_rna1_at      0.987377
U82275_at           0.991114
U41813_at           0.993868
Y00787_s_at         1.000000
X95735_at           1.000000
U80457_at           1.000000
L27584_s_at         1.000000
U32315_at           1.000000
U22376_cds2_s_at    1.000000
HG2788-HT2896_at    1.000000
dtype: float64

In [30]:
phik_imp_features = phik_cors[phik_cors > 0.75].index
phik_imp_features


Index(['AFFX-CreX-5_st', 'AFFX-DapX-5_at', 'AFFX-HUMGAPDH/M33197_3_at',
       'AFFX-HUMTFRR/M11507_5_at', 'AFFX-HUMTFRR/M11507_M_at',
       'AFFX-HUMTFRR/M11507_3_at', 'A28102_at', 'AB000467_at', 'AB002382_at',
       'AB006190_at',
       ...
       'L32961_at', 'U20499_at', 'U62434_at', 'X06318_at', 'X51345_at',
       'D38437_f_at', 'J00212_f_at', 'M37755_f_at', 'L10717_at', 'U29175_at'],
      dtype='object', length=454)

In [41]:
# pd.Series(phik_imp_features).to_csv("phik_important.csv")

In [ ]:
cors = X_train.apply(lambda x: x.corr(target_one_hot))


In [ ]:
# persons.join(target)


In [31]:
# cors = cors.sort_values()
# cors


In [ ]:
important_features = cors[cors.abs() > 0.7].index


In [ ]:
# cors.iloc[:30]


In [ ]:
# cors.iloc[-15:]


In [64]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


In [23]:
# X_train = X_train[phik_imp_features]
# X_val = X_val[phik_imp_features]
# X_test = X_test[phik_imp_features]


In [52]:
# X_train = pd.concat([X_train, get_dummies()], axis=1)
# X_test = pd.concat([X_train, get_dummies(X_test)], axis=1)
# X_val = pd.concat([X_val, get_dummies(X_val)], axis=1)

In [65]:
reg = LogisticRegression()
reg.fit(X_train, Y_train)
predicted = reg.predict(X_val)


In [66]:
predicted


array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1])

In [67]:
from sklearn.metrics import classification_report


In [68]:
print(classification_report(Y_val, predicted))


              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.92      1.00      0.96        12

    accuracy                           0.94        16
   macro avg       0.96      0.88      0.91        16
weighted avg       0.94      0.94      0.93        16

